# 1. DB연결 & 스키마 생성 코드

In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

from dataManageModule.db_connection import get_connection

# 1. DB 파일 경로 설정
DB_PATH = "FDA-study.db"

# 2. 연결 생성
conn = get_connection(DB_PATH)
cursor = conn.cursor()

# 3. 스키마 생성 (개념 → 유형 → 문제)
schema_sql = """
PRAGMA foreign_keys = ON; -- sqlite에 외래키 제약 설정

-- 개념 테이블
CREATE TABLE IF NOT EXISTS concepts (
    concept_id      INTEGER PRIMARY KEY AUTOINCREMENT,
    name            TEXT NOT NULL,                                   -- 개념 이름 (ex. "정규분포")
    level           INTEGER NOT NULL CHECK(level BETWEEN 1 AND 5),   -- 개념 레벨은 1-5 사이로만 설정
    parent_id       INTEGER,
    note_path       TEXT,
    FOREIGN KEY (parent_id) REFERENCES concepts(concept_id)
);

-- 유형 테이블
CREATE TABLE IF NOT EXISTS types (
    type_id         INTEGER PRIMARY KEY AUTOINCREMENT,
    concept_id      INTEGER NOT NULL,
    name            TEXT NOT NULL,                                   -- 유형 이름 (ex. "정규분포의 평균/분산 계산")
    note_path       TEXT,
    FOREIGN KEY (concept_id) REFERENCES concepts(concept_id)
);

-- 문제 테이블
CREATE TABLE IF NOT EXISTS problems (
    problem_id              INTEGER PRIMARY KEY AUTOINCREMENT,
    year                    INTEGER,                       -- 기출연도
    number                  INTEGER,                       -- 문제번호
    type_id                 INTEGER NOT NULL,              -- 대표 유형 FK
    note_path       TEXT,                          -- 문제본문
    FOREIGN KEY (type_id) REFERENCES types(type_id)
);
"""

cursor.executescript(schema_sql)
conn.commit()

print("테이블 생성 완료 ✅")


테이블 생성 완료 ✅


# 2. DB 읽어오기

In [4]:
from dataManageModule.db_connection import get_connection
from dataManageModule.crud_ops import fetch_concepts, fetch_types, fetch_problems
import json

conn = get_connection()

rows = fetch_concepts(conn)

for row in rows:
    print(json.dumps(dict(row), ensure_ascii=False, indent=2))


NameError: name 'Optional' is not defined

# 3. DB 수정하기

In [ ]:
import ipywidgets as widgets
import dataManageModule.crud_ui as crud_ui

tab_titles = ["New", "Search"]

tab_boxes = {
    title: widgets.VBox([]) for title in tab_titles
}

tab_boxes["Search"].children = [
    crud_ui.search_control,
    crud_ui.search_result,
    crud_ui.search_detail,
]

tabs = widgets.Tab(children=list(tab_boxes.values()))

for i, title in enumerate(tab_titles):
    tabs.set_title(i, title)


display(tabs)